In [ ]:
import phoenix as px
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

In [ ]:
PostgresSession = sessionmaker(
    create_engine(
        "postgresql+psycopg://localhost:5432/postgres?user=postgres&password=mysecretpassword",
        echo=True,
    ),
    expire_on_commit=False,
)
SqliteSession = sessionmaker(
    create_engine("sqlite:////Users/xandersong/.phoenix/phoenix.db", echo=True),
    expire_on_commit=False,
)

In [ ]:
original_endpoint = "http://127.0.0.1:6008"
sqlite_endpoint = "http://127.0.0.1:6006"
postgres_endpoint = "http://127.0.0.1:6007"

In [ ]:
COMMON_COLUMNS = [
    "attributes.__computed__",
    "attributes.input.value",
    "attributes.openinference.span.kind",
    "attributes.output.value",
    "context.span_id",
    "context.trace_id",
    "end_time",
    "events",
    "name",
    "parent_id",
    "start_time",
    "status_code",
    "status_message",
]


def get_spans_dataframe(endpoint: str, filter_condition: str):
    df = px.Client(endpoint=endpoint).get_spans_dataframe(filter_condition)
    return df.sort_index()

In [ ]:
filter_condition = "evals['Q&A Correctness'].label == 'correct'"
original_df = get_spans_dataframe(endpoint=original_endpoint, filter_condition=filter_condition)
postgres_df = get_spans_dataframe(endpoint=postgres_endpoint, filter_condition=filter_condition)
sqlite_df = get_spans_dataframe(endpoint=sqlite_endpoint, filter_condition=filter_condition)
print(f"{original_df.shape=}")
print(f"{postgres_df.shape=}")
print(f"{sqlite_df.shape=}")

In [ ]:
sqlite_df[COMMON_COLUMNS].compare(
    original_df.rename(columns={"span_kind": "attributes.openinference.span.kind"})[COMMON_COLUMNS],
    result_names=("sqlite", "original"),
)

In [ ]:
filter_condition = (
    """evals['Q&A Correctness'].label == 'correct' and evals["Hallucination"].score < 0.5"""
)
original_df = get_spans_dataframe(endpoint=original_endpoint, filter_condition=filter_condition)
postgres_df = get_spans_dataframe(endpoint=postgres_endpoint, filter_condition=filter_condition)
sqlite_df = get_spans_dataframe(endpoint=sqlite_endpoint, filter_condition=filter_condition)
print(f"{original_df.shape=}")
print(f"{postgres_df.shape=}")
print(f"{sqlite_df.shape=}")

In [ ]:
sqlite_df[COMMON_COLUMNS].compare(
    original_df.rename(columns={"span_kind": "attributes.openinference.span.kind"})[COMMON_COLUMNS],
    result_names=("sqlite", "original"),
)